<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/opt_model_finetune_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import transformers

print(transformers.__version__)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

4.22.2


In [3]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'val.txt'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-57edfef27031067a/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(datasets["train"])

In [6]:
#"facebook/opt-125m"
model_checkpoint = "facebook/opt-125m"

In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/651 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
# block_size = tokenizer.model_max_length
block_size = 128

In [11]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

In [14]:
from transformers import Trainer, TrainingArguments

In [15]:
#https://stackoverflow.com/questions/72680734/huggingface-trainer-only-doing-3-epochs-no-matter-the-trainingarguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-health-test-1",
    evaluation_strategy = "steps", #epoch
    num_train_epochs=6, 
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    warmup_steps=1000,                # number of warmup steps for learning rate  
    save_steps=2000,
    eval_steps=500,
    save_total_limit=10,
    load_best_model_at_end= True,
    #output_dir='./results',
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [17]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32663
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 24498


Step,Training Loss,Validation Loss
500,3.115400,2.968299
1000,2.987800,2.916107
1500,2.957800,2.878906
2000,2.909700,2.853606
2500,2.877800,2.834317
3000,2.856500,2.815716
3500,2.836700,2.806188
4000,2.821700,2.790994
4500,2.703800,2.785455
5000,2.676200,2.778279


***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
Saving model checkpoint to opt-125m-health-test-1/checkpoint-2000
Configuration saved in opt-125m-health-test-1/checkpoint-2000/config.json
Model weights saved in opt-125m-health-test-1/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4387
  Batch size = 8
Saving model checkpoint to opt-125m-health-test-1/checkpoint-4000
Configuration saved in opt-125m-health-test-1/checkpoint-4000/config.json
Model weights saved in opt-125m-health-test-1/checkpoint-4000/pytorch_model.bin


TrainOutput(global_step=24498, training_loss=2.5156772503590856, metrics={'train_runtime': 5619.4948, 'train_samples_per_second': 34.875, 'train_steps_per_second': 4.359, 'total_flos': 1.2801872461824e+16, 'train_loss': 2.5156772503590856, 'epoch': 6.0})

In [26]:
from transformers import AutoModelForCausalLM

#tokenizer2 = AutoTokenizer.from_pretrained("opt-125m-health-test-1/checkpoint-24000", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("opt-125m-health-test-1/checkpoint-24000")

loading configuration file opt-125m-health-test-1/checkpoint-24000/config.json
Model config OPTConfig {
  "_name_or_path": "opt-125m-health-test-1/checkpoint-24000",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading weights file opt-125m-health-test-1/checkpoint-24000/pytorch_model.bin
All model checkpoint weights were used when initializing OPTForCausalLM.

All the weights of OPTForCausalLM were 

In [28]:
import importlib
import generate_txt
importlib.reload(generate_txt)
generator = generate_txt.generate_model_results(tokenizer, model)

In [29]:
test_metrics_1 = generator.generate_sample("The event has a ", visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [31]:
for key in test_metrics_1.keys():
  print(key)

The event has a 
 root mean square error of ~12%, and has been estimated to be at 0% to 50% with no data reporting in the paper. The error can be improved with a series of experiments. As an estimation, we found that the basic reproduction number E0(t) that gives the disease control is approximately 2.
 long delay with a mean estimate of 0.03 days and 95% confidence intervals (CI).
 score which can be used to predict the future. For the past 10 days the daily number of infected people in Korea has nearly doubled and the last day of the doubling-peak indicates that the trend is accelerating. The current case load is almost 10 million as of 6 days after the beginning of the epidemic. A more accurate
 range between. The expected mortality may be, 1-0.5 per 1000 and.. The expected mortality may be,... The estimated time from the start of the outbreak to a country. The estimated time from the start of infection to death and the total number of infections in the country. The estimated total


In [34]:
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['event'], 'number_sents': 1, 'avg_sent_len': 4.0, 'sent_repeat_score': [0.0]}
{'number_ents': 2, 'ents': [0% to 50%, 2], 'number_nouns': 16, 'nouns': ['root', 'error', '%', '%', '%', 'datum', 'paper', 'error', 'series', 'experiment', 'estimation', 'reproduction', 'number', 'E0(t', 'disease', 'control'], 'number_sents': 4, 'avg_sent_len': 15.5, 'sent_repeat_score': [0.0, 0.1428571428571429, 0.0, 0.09090909090909094]}
{'number_ents': 3, 'ents': [0.03 days, 95%, CI], 'number_nouns': 7, 'nouns': ['delay', 'estimate', 'day', '%', 'confidence', 'interval', 'CI'], 'number_sents': 1, 'avg_sent_len': 19.0, 'sent_repeat_score': [0.0]}
{'number_ents': 5, 'ents': [the past 10 days, Korea, the last day, almost 10 million, 6 days], 'number_nouns': 15, 'nouns': ['score', 'future', 'day', 'number', 'people', 'Korea', 'day', 'doubling', 'peak', 'trend', 'case', 'load', 'day', 'beginning', 'epidemic'], 'number_sents': 4, 'avg_sent_len': 16.25, 

In [37]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric']['number_ents'], test_metrics_1[key]['metric']['number_nouns'],
        test_metrics_1[key]['metric']['number_sents'],  test_metrics_1[key]['metric']['avg_sent_len'],
        test_metrics_1[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 4.0 [0.0]
2 16 4 15.5 [0.0, 0.1428571428571429, 0.0, 0.09090909090909094]
3 7 1 19.0 [0.0]
5 15 4 16.25 [0.0, 0.1875, 0.1578947368421053, 0.0]
0 14 6 10.0 [0.0, 0.0, 0.0, 0.15384615384615385, 0.25, 0.0]
0 17 2 29.0 [0.04166666666666663, 0.20588235294117652]


In [38]:
#generator("The wedding started with a ")

In [39]:
test_metrics_2 = generator.generate_sample("The conference started with a ", visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [40]:
for key in test_metrics_2.keys():
  print(key)

The conference started with a 
 talk by Dr Hsu and he called on health system practitioners to make plans for treating COVID-19 patients. Following his talk, he asked healthcare practitioners what they were hoping to achieve by identifying the early signs and treating the patients with appropriate therapies. The conference was successful, and both the National Health Commission of China and
 panel discussion on the rapid changes seen in the current epidemics. Experts including statisticians, statisticians, medical statisticians, epidemiologists and other specialists addressed the issue of the predictions for COVID-19 epidemic in Italy. The  panel discussed the predictions for the epidemic of 2019-nCoV to date.
 question and a response period that lasted a few minutes. The  Questionnaire asked a person about the health conditions affecting them, including: whether it was a year and a half since their last visit, whether or not it had already become clear that they were on a healthy diet

In [41]:
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['conference'], 'number_sents': 1, 'avg_sent_len': 5.0, 'sent_repeat_score': [0.0]}
{'number_ents': 2, 'ents': [Hsu, the National Health Commission of China], 'number_nouns': 20, 'nouns': ['talk', 'Dr', 'Hsu', 'health', 'system', 'practitioner', 'plan', 'covid-19', 'patient', 'talk', 'healthcare', 'practitioner', 'sign', 'patient', 'therapy', 'conference', 'National', 'Health', 'Commission', 'China'], 'number_sents': 3, 'avg_sent_len': 20.333333333333332, 'sent_repeat_score': [0.0, 0.03703703703703709, 0.1428571428571429]}
{'number_ents': 2, 'ents': [Italy, 2019-nCoV], 'number_nouns': 20, 'nouns': ['panel', 'discussion', 'change', 'epidemic', 'expert', 'statistician', 'statistician', 'statistician', 'epidemiologist', 'specialist', 'issue', 'prediction', 'covid-19', 'epidemic', 'Italy', 'panel', 'prediction', 'epidemic', 'nCoV', 'date'], 'number_sents': 4, 'avg_sent_len': 13.5, 'sent_repeat_score': [0.0, 0.08333333333333337, 0.1

In [42]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric']['number_ents'], test_metrics_2[key]['metric']['number_nouns'],
        test_metrics_2[key]['metric']['number_sents'],  test_metrics_2[key]['metric']['avg_sent_len'],
        test_metrics_2[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 5.0 [0.0]
2 20 3 20.333333333333332 [0.0, 0.03703703703703709, 0.1428571428571429]
2 20 4 13.5 [0.0, 0.08333333333333337, 0.19999999999999996, 0.125]
2 13 3 21.333333333333332 [0.0, 0.09090909090909094, 0.25]
0 19 4 14.0 [0.0, 0.0, 0.3555555555555555, 0.0]
1 13 3 18.666666666666668 [0.0, 0.21212121212121215, 0.09090909090909094]


In [43]:
#event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '


In [45]:
test_metrics_3 = generator.generate_sample(sick_txt, visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [47]:
for key in test_metrics_3.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
 not  take all months. Hopefully this year, the spread will be much closer than in 2018, because all our teams will be active all the time. But I am very anxious, but we have good chemistry. I am not much help. For some reason, this virus always changes its behaviour, and this year it
 be my last year to attend this year. I will be at NYSC and you were all excellent this year. My name is Stephen, and today, I am a nurse, so this is a lot to ask for. However, you are probably reading this already. What has been on your mind over the past few
 make us very cautious. Also, the conference is in London, U. K., so we have to be here. On the other hand, the flu and the vaccine are a two-part thing because you have to really be vaccinated before going in and getting your vaccinations. Before you are in the Conference Center, just
 give you an additional opportunity to participate.

In [48]:
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 4, 'ents': [all months, this year, 2018, this year], 'number_nouns': 11, 'nouns': ['month', 'year', 'spread', 'team', 'time', 'chemistry', 'help', 'reason', 'virus', 'behaviour', 'year'], 'number_sents': 6, 'avg_sent_len': 10.833333333333334, 'sent_repeat_score': [0.1428571428571429, 0.19999999999999996, 0.08333333333333337, 0.0, 0.1428571428571429, 0.0]}
{'number_ents': 5, 'ents': [last year, this year, this year, Stephen, today], 'number_nouns': 10, 'nouns': ['year', 'year', 'NYSC', 'year', 'name', 'Stephen', 'today', 'nurse', 'lot', 'mind'], 'number_sents': 5, 'avg_sent_len': 12.8, 'sent_repeat_score': [0.09999999999999998, 0.07692307692307687, 0.2272727272727273, 0.0, 0.0]}
{'number_ents': 3, 'ents': [London, two, the Conference Center], 'number_nouns': 12, 'nouns': ['conference', 

In [49]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric']['number_ents'], test_metrics_3[key]['metric']['number_nouns'],
        test_metrics_3[key]['metric']['number_sents'],  test_metrics_3[key]['metric']['avg_sent_len'],
        test_metrics_3[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
4 11 6 10.833333333333334 [0.1428571428571429, 0.19999999999999996, 0.08333333333333337, 0.0, 0.1428571428571429, 0.0]
5 10 5 12.8 [0.09999999999999998, 0.07692307692307687, 0.2272727272727273, 0.0, 0.0]
3 12 5 12.8 [0.0, 0.0, 0.16666666666666663, 0.12903225806451613, 0.0]
2 3 3 10.666666666666666 [0.0, 0.0, 0.0]
2 11 5 12.8 [0.1428571428571429, 0.07407407407407407, 0.0, 0.1428571428571429, 0.0]


In [50]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
#generator2(event_txt + sick_txt)
combo = event_txt + sick_txt

In [51]:
test_metrics_4 = generator.generate_sample(combo, visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [52]:
for key in test_metrics_4.keys():
  print(key)

Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
 be the third time this year that the event has seen such attention. It may also be an opportunity (albeit only) to show how climate change (also known as the flu) might impact people of all ages. And to put it simply: people do think, and the media has heard it all. Climate disruption?
 be the start of an unprecedented year for climate and ecology. We will bring you new research & updates. Stay informed with live updates on the current COVID-19 outbreak and visit our coronavirus hub for more advice on prevention and treatment. As if environmenta

In [53]:
for key in test_metrics_4.keys():
  print(test_metrics_4[key]['metric'])

{'number_ents': 2, 'ents': [three, this year], 'number_nouns': 23, 'nouns': ['Climate', 'Warriors', 'youth', 'Climate', 'Justice', 'series', 'webinar', 'climate', 'justice', 'initiative', 'community', 'organizer', 'movement', 'community', 'way', 'legislator', 'charge', 'climate', 'justice', 'flu', 'year', 'conference', 'vaccine'], 'number_sents': 6, 'avg_sent_len': 12.333333333333334, 'sent_repeat_score': [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]}
{'number_ents': 3, 'ents': [third, this year, all ages], 'number_nouns': 14, 'nouns': ['time', 'year', 'event', 'attention', 'opportunity', 'climate', 'change', 'flu', 'people', 'age', 'people', 'medium', 'climate', 'disruption'], 'number_sents': 4, 'avg_sent_len': 16.25, 'sent_repeat_score': [0.06666666666666665, 0.10344827586206895, 0.11111111111111116, 0.0]}
{'number_ents': 1, 'ents': [an unprecedented year], 'number_nouns': 18, 'nouns': ['start', 'year', 'climate', 'ecology', 'research', 'update', 'update', 'covid-19', 'outbreak', 'c

In [54]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_4.keys():
  metric_load = test_metrics_4[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
2 23 6 12.333333333333334 [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]
3 14 4 16.25 [0.06666666666666665, 0.10344827586206895, 0.11111111111111116, 0.0]
1 18 4 14.5 [0.0, 0.0, 0.08695652173913049, 0.07692307692307687]
2 8 4 10.75 [0.0, 0.0, 0.0, 0.25]
1 12 4 11.5 [0.050000000000000044, 0.0, 0.0, 0.0]
3 14 4 15.75 [0.0, 0.125, 0.06896551724137934, 0.19999999999999996]


In [55]:
#check different lengths
length_metric_1 = generator.generate_sample(sick_txt, max_length=32, visualize=False)
length_metric_2 = generator.generate_sample(sick_txt, max_length=64, visualize=False)
length_metric_3 = generator.generate_sample(sick_txt, max_length=128, visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [67]:
for key in length_metric_1.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
 make 2020 better.
18
 not affect the daily life of people. This may change before 2020, and in some cases, it will change after that. So please. The symptoms were very
146
 prevent you from getting the flu, but they won’t prevent you from getting the flu. Don’t worry. The year can pass before you
125
 be a chance to tell the public all about the flu. However, we need to see for ourselves the effects of all this, so don’t be
125
 be a bad way for everyone. This year will be very tough. We have to pay very close attention. The flu is also a problem because we have a
138


In [57]:
for key in length_metric_1.keys():
  print(length_metric_1[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 1, 'ents': [2020], 'number_nouns': 0, 'nouns': [], 'number_sents': 2, 'avg_sent_len': 2.5, 'sent_repeat_score': [0.0, 0.0]}
{'number_ents': 2, 'ents': [daily, 2020], 'number_nouns': 4, 'nouns': ['life', 'people', 'case', 'symptom'], 'number_sents': 4, 'avg_sent_len': 8.25, 'sent_repeat_score': [0.0, 0.11764705882352944, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 3, 'nouns': ['flu', 'flu', 'year'], 'number_sents': 4, 'avg_sent_len': 7.25, 'sent_repeat_score': [0.0, 0.33333333333333337, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 4, 'nouns': ['chance', 'public', 'flu', 'effect'], 'number_sents': 2, 'avg_sent_len': 15.5, 'sent_repeat_score': [0.07692307692307687, 0.05555555555555558]}
{'number_ents': 1, 'ents': [This year], 'number_nouns': 5, 'nouns': ['way', 

In [58]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_1.keys():
  metric_load = length_metric_1[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
1 0 2 2.5 [0.0, 0.0]
2 4 4 8.25 [0.0, 0.11764705882352944, 0.0, 0.0]
0 3 4 7.25 [0.0, 0.33333333333333337, 0.0, 0.0]
0 4 2 15.5 [0.07692307692307687, 0.05555555555555558]
1 5 5 6.6 [0.0, 0.0, 0.0, 0.0, 0.0]


In [59]:
for key in length_metric_2.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
 affect me in December, 2020. No harm, no foul. But, this year, it won’t be easy. To be honest, I’m not really sure what to expect. For example, I’m very concerned about COVID-19. While I know that there are a lot
 be a major event. Also to help us make sure that the world is well prepared, I would like to ask you to join me tomorrow for a presentation. And have a great day!Alain Blanc, CEO of WellRx. com
 be a very tough year, everyone will have had a lot of training, some doctors with sick days, some people from all the countries of the world, and it will be an unprecedented year. As our daily living is very difficult with many situations, it is also really hard in many situations to prepare. So, at
 make us ready for a new year. This year I hope we will have more practical and practical experiences for every one year. A great many people have left us behind this year. M

In [66]:
for key in length_metric_2.keys():
    print(key)
    print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
 affect me in December, 2020. No harm, no foul. But, this year, it won’t be easy. To be honest, I’m not really sure what to expect. For example, I’m very concerned about COVID-19. While I know that there are a lot
213
 be a major event. Also to help us make sure that the world is well prepared, I would like to ask you to join me tomorrow for a presentation. And have a great day!Alain Blanc, CEO of WellRx. com
194
 be a very tough year, everyone will have had a lot of training, some doctors with sick days, some people from all the countries of the world, and it will be an unprecedented year. As our daily living is very difficult with many situations, it is also really hard in many situations to prepare. So, at
301
 make us ready for a new year. This year I hope we will have more practical and practical experiences for every one year. A great many people have left us beh

In [61]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_2.keys():
  metric_load = length_metric_2[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
3 7 7 8.0 [0.0, 0.0, 0.16666666666666663, 0.09090909090909094, 0.07692307692307687, 0.0, 0.0]
2 8 4 11.25 [0.0, 0.11111111111111116, 0.0, 0.0]
2 12 3 21.666666666666668 [0.275, 0.13636363636363635, 0.0]
5 10 6 10.666666666666666 [0.0, 0.0, 0.11764705882352944, 0.0, 0.08333333333333337, 0.1333333333333333]
1 11 5 12.2 [0.0, 0.11764705882352944, 0.09090909090909094, 0.0, 0.18181818181818177]


In [65]:
for key in length_metric_3.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
 make me sound like a dumbass. You all are crazy. All of the professionals are dying this year, this is why we’re being told to stay at home. But, as far as I know, if you’re sick, people are taking advantage of this, and we don’t have enough vaccine or treatments. This is something I’ve been wanting to do for decades. But now this, this is something else entirely. This is something I just don’t understand. It’s like getting another flu shot at the age of 50 years. You would be making the same choices
506
 not be just a scientific research crisis, but also a spiritual crisis. In this crisis, people are seeking healing from the world as they know that God was present in their life, and from the point of view of the flu virus. In addition, people are facing spiritual imbalances that leave them unable to eat without fasting, sleep, or take a drug. Most people with flu, ev

In [63]:
for key in length_metric_3.keys():
  print(length_metric_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 3, 'ents': [this year, decades, the age of 50 years], 'number_nouns': 14, 'nouns': ['dumbass', 'professional', 'year', 'home', 'people', 'advantage', 'vaccine', 'treatment', 'decade', 'flu', 'shot', 'age', 'year', 'choice'], 'number_sents': 10, 'avg_sent_len': 11.6, 'sent_repeat_score': [0.0, 0.0, 0.0, 0.1428571428571429, 0.1333333333333333, 0.08333333333333337, 0.09999999999999998, 0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 26, 'nouns': ['research', 'crisis', 'crisis', 'crisis', 'people', 'healing', 'world', 'God', 'life', 'point', 'view', 'flu', 'virus', 'addition', 'people', 'imbalance', 'sleep', 'drug', 'people', 'flu', 'imbalance', 'imbalance', 'aspect', 'life', 'example', 'flu'], 'number_sents': 6, 'avg_sent_len': 20.166666666666668, 'sent_repeat_score': [0.13

In [64]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_3.keys():
  metric_load = length_metric_3[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
3 14 10 11.6 [0.0, 0.0, 0.0, 0.1428571428571429, 0.1333333333333333, 0.08333333333333337, 0.09999999999999998, 0.0, 0.0, 0.0]
0 26 6 20.166666666666668 [0.1333333333333333, 0.21212121212121215, 0.08333333333333337, 0.1785714285714286, 0.07692307692307687, 0.0]
0 2 3 5.333333333333333 [0.0, 0.0, 0.0]
0 18 9 14.0 [0.0, 0.17647058823529416, 0.0714285714285714, 0.08695652173913049, 0.11764705882352944, 0.07407407407407407, 0.2142857142857143, 0.0, 0.0]
4 12 5 13.6 [0.0, 0.0, 0.0, 0.13043478260869568, 0.07692307692307687]


In [ ]:
#https://huggingface.co/docs/transformers/v4.16.2/en/custom_datasets

In [21]:
!cp -r opt-125m-health-test-1/checkpoint-24000/ opt-125m-health-test-v2/

In [22]:
!zip -r opt_health_v1.zip opt-125m-health-test-v2/

  adding: opt-125m-health-test-v2/ (stored 0%)
  adding: opt-125m-health-test-v2/checkpoint-24000/ (stored 0%)
  adding: opt-125m-health-test-v2/checkpoint-24000/rng_state.pth (deflated 27%)
  adding: opt-125m-health-test-v2/checkpoint-24000/pytorch_model.bin (deflated 8%)
  adding: opt-125m-health-test-v2/checkpoint-24000/scheduler.pt (deflated 49%)
  adding: opt-125m-health-test-v2/checkpoint-24000/trainer_state.json (deflated 83%)
  adding: opt-125m-health-test-v2/checkpoint-24000/config.json (deflated 48%)
  adding: opt-125m-health-test-v2/checkpoint-24000/training_args.bin (deflated 48%)
  adding: opt-125m-health-test-v2/checkpoint-24000/optimizer.pt (deflated 11%)
  adding: opt-125m-health-test-v2/.ipynb_checkpoints/ (stored 0%)


In [23]:
from google.colab import files
files.download("opt_health_v1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>